In [16]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

In [17]:
impute_and_scale = False

if impute_and_scale:
    np.random.seed(0)
    df = pd.read_csv('processed_dataset.csv')

    df = df.loc[(df['homeStatus'] <= 2)]
    df = df.loc[(df['price'] > 100)]

    # split into train and test
    test_ratio = 0.2
    train_indices = np.sort(np.random.choice(df.shape[0], int((1-test_ratio)*df.shape[0]), replace=False))
    test_indices = np.setdiff1d(np.arange(df.shape[0]), train_indices)

    #train_indices = train_indices[:int(len(train_indices))]
    #test_indices = test_indices[:int(len(test_indices))]

    X_train = X.iloc[train_indices]
    y_train = y.iloc[train_indices]
    X_test = X.iloc[test_indices]
    y_test = y.iloc[test_indices]

    imputer = KNNImputer(n_neighbors=5)
    X_train = imputer.fit_transform(X_train)
    X_test = imputer.transform(X_test)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    df.to_csv('imputed_scaled_dataset_0.csv', index=True)
else:
    df = pd.read_csv('imputed_scaled_dataset_0.csv')

In [18]:
y = df['price']
X = df.drop('price', axis=1)

# split into train and test
np.random.seed(0)
test_ratio = 0.2
train_indices = np.sort(np.random.choice(df.shape[0], int((1-test_ratio)*df.shape[0]), replace=False))
test_indices = np.setdiff1d(np.arange(df.shape[0]), train_indices)

X_train = X.iloc[train_indices]
y_train = y.iloc[train_indices]
X_test = X.iloc[test_indices]
y_test = y.iloc[test_indices]